# Introduction to LangGraph 🕸️

The LangGraph [quick start](https://langchain-ai.github.io/langgraph/tutorials/introduction/#quick-start) was leveraged to build this example.

In [ ]:
from typing import Annotated
import os
from dotenv import load_dotenv
load_dotenv()
from typing_extensions import TypedDict
from langchain_openai import AzureChatOpenAI
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from IPython.display import Image, display
from langchain_community.tools.tavily_search import TavilySearchResults

## Define Azure OpenAI GPT4o LLM 🤖

We will be leveraging GPT4o and the main LLM in our agentic application.

In [ ]:
llm = AzureChatOpenAI(api_key=os.getenv("AZURE_OPENAI_API_KEY"), azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"), deployment_name="gpt4o", api_version="2024-03-01-preview")

## Define [Tavily Search](https://app.tavily.com/sign-in) Tool 🔨

Tavily search is an interesting tool that claims to be optimized for 'AI agents and LLM's'. They have some details on their [website](https://docs.tavily.com/docs/tavily-api/introduction#why-choose-tavily-search-api) as to 'why' to go with their API for search.

In [ ]:
tool = TavilySearchResults(max_results=2)
tools = [tool]
tool.invoke("What's a 'node' in LangGraph?")

## Create StateGraph

This class will help manage the state of the agentic application

In [ ]:
class State(TypedDict):
    messages: Annotated[list, add_messages]
    

graph_builder = StateGraph(State)
llm_with_tools = llm.bind_tools(tools)


def chatbot(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}


graph_builder.add_node("chatbot", chatbot)

## Create BasicToolNode

The below class will check the most recent message in the state and call's the tool(s) the application has access to if the message contains tool_calls. Note, it relies on an LLM's tool calling support, which, in our case, OpenAI [supports](https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/function-calling).

You will notice we add the Tavily Search tool we defined earlier to the agents 'tool belt'

In [ ]:
import json

from langchain_core.messages import ToolMessage


class BasicToolNode:
    """A node that runs the tools requested in the last AIMessage."""

    def __init__(self, tools: list) -> None:
        self.tools_by_name = {tool.name: tool for tool in tools}

    def __call__(self, inputs: dict):
        if messages := inputs.get("messages", []):
            message = messages[-1]
        else:
            raise ValueError("No message found in input")
        outputs = []
        for tool_call in message.tool_calls:
            tool_result = self.tools_by_name[tool_call["name"]].invoke(
                tool_call["args"]
            )
            outputs.append(
                ToolMessage(
                    content=json.dumps(tool_result),
                    name=tool_call["name"],
                    tool_call_id=tool_call["id"],
                )
            )
        return {"messages": outputs}


tool_node = BasicToolNode(tools=[tool])
graph_builder.add_node("tools", tool_node)

## Define route_tools function

This helps tell the graph wether to select a tool or complete the loop.

In [ ]:
from typing import Literal


def route_tools(
    state: State,
) -> Literal["tools", "__end__"]:
    """
    Use in the conditional_edge to route to the ToolNode if the last message
    has tool calls. Otherwise, route to the end.
    """
    if isinstance(state, list):
        ai_message = state[-1]
    elif messages := state.get("messages", []):
        ai_message = messages[-1]
    else:
        raise ValueError(f"No messages found in input state to tool_edge: {state}")
    if hasattr(ai_message, "tool_calls") and len(ai_message.tool_calls) > 0:
        return "tools"
    return "__end__"
graph_builder.add_conditional_edges(
    "chatbot",
    route_tools,
    {"tools": "tools", "__end__": "__end__"},
)
graph_builder.add_edge("tools", "chatbot")
graph_builder.add_edge(START, "chatbot")
graph = graph_builder.compile()

## Displays the Mermaid Diagram for the Graph 🧜

Visually displays how the graph routes to various nodes.

In [ ]:
from IPython.display import Image, display

try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception:
    pass

## Infinite Chat Loop ♾️

Chat loop for testing the agentic application.

In [ ]:
from langchain_core.messages import BaseMessage

while True:
    user_input = input("User: ")
    if user_input.lower() in ["quit", "exit", "q"]:
        break
    for event in graph.stream({"messages": [("user", user_input)]}):
        for value in event.values():
            if isinstance(value["messages"][-1], BaseMessage):
                print("Assistant:", value["messages"][-1].content)